In [3]:
#pip install ta


In [5]:
import numpy as np
import pandas as pd
from ta.trend import SMAIndicator
from ta.trend import ADXIndicator
from ta.trend import CCIIndicator
from ta.momentum import RSIIndicator
import matplotlib.pyplot as plt
import matplotlib.style
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")
plt.style.use("ggplot")

In [6]:
# Símbolo de la acción que deseas descargar (por ejemplo, "AAPL" para Apple)
symbol = "AAPL"

# Rango de fechas para los datos (por ejemplo, desde el 1 de enero de 2020 hasta el 31 de diciembre de 2020)
start_date = "2020-01-01"
end_date = "2020-12-31"

# Descargar los datos utilizando la función Ticker de yfinance
df = yf.download(symbol, start=start_date, end=end_date)

# Reordenar las columnas
df = df[["Open", "High", "Low", "Close", "Volume"]]

# Renombrar las columnas
df.columns = ["Price", "maximo", "minimo", "cierre", "volumen"]



# Reiniciar el índice para que la columna "Date" sea una columna regular
df = df.reset_index()

df = pd.DataFrame(df)

df["Date"] = pd.to_datetime(df["Date"])

[*********************100%%**********************]  1 of 1 completed


In [7]:
        # Inicializar variables necesarias
precio_compra = 0
cantidad_comprada = 0
precio_maximo = 0
fecha_compra = None
operacion = 1




trades_data = pd.DataFrame(columns=['Operacion', 'Fecha de Compra', 'Precio de Compra', 'Cantidad Comprada',"Capital Inicial",
                                    'Fecha de Venta',"Tipo de venta", 'Precio de Venta',"Capital Final", 'Días de Operación', 'Resultado', 'Resultado Porcentual'])

periodo = 14


capital_fijo=1000
capital_inicial = 1000
acciones_inicial = 0.0
capital = capital_inicial
acciones = acciones_inicial
comisiones = 0.005
# Porcentajes para las estrategias
porcentaje_stop_loss = 1  # Termina la operación si se pierde un 10%
porcentaje_take_profit = 10  # Termina la operación si se gana un 20%
porcentaje_trailing_stop = 1  # Termina la operación si se cae un 5% desde el precio más alto





In [8]:
class Estrategia():

    LONG_MEDIA_VALUE = periodo


    def aplicar(self, df):
        df = self.analisis(df)
        return self.__estrategia_adx(df)

    def analisis(self, df,):
        iADX = ADXIndicator(df['maximo'], df['minimo'], df['cierre'], Estrategia.LONG_MEDIA_VALUE, False)
        df['ADX'] = iADX.adx()
        df['ADX+'] = iADX.adx_pos()
        df['ADX-'] = iADX.adx_neg()
        return df

    def __estrategia_adx(self, df):
        df['Posicion'] = np.where(np.logical_and(df['ADX-'] < df['ADX+'], (df['ADX+'] - df['ADX-']) > 1), 'Compra', 'Venta')
        df['Alternancia'] = df['Posicion'].ne(df['Posicion'].shift())
        df['Orden'] = np.where(df['Alternancia'], df['Posicion'], '')
        df['Orden_Precio'] = np.where(df['Orden'].isin(['Compra', 'Venta']), df['Price'], '')
        return df


In [9]:
estrategia = Estrategia()  # Crear una instancia de la clase Estrategia
resultado = estrategia.aplicar(df)  # Llamar al método aplicar() en la instancia y pasar df como argumento


In [10]:
df

,Date,Price,maximo,minimo,cierre,volumen,ADX,ADX+,ADX-,Posicion,Alternancia,Orden,Orden_Precio
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,135480400,0.000000,0.000000,0.000000,Venta,True,Venta,74.05999755859375
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,146322800,0.000000,0.000000,0.000000,Venta,False,,
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,118387200,0.000000,0.000000,0.000000,Venta,False,,
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,108872000,0.000000,0.000000,0.000000,Venta,False,,
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,132079200,0.000000,0.000000,0.000000,Venta,False,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-12-23,132.160004,132.429993,130.779999,130.960007,88223700,19.541019,34.533669,18.712291,Compra,False,,
248,2020-12-24,131.320007,133.460007,131.100006,131.970001,54930100,20.490430,34.911115,17.652895,Compra,False,,
249,2020-12-28,133.990005,137.339996,133.509995,136.690002,124486200,22.103937,39.235249,15.608724,Compra,False,,
250,2020-12-29,138.050003,138.789993,134.339996,134.869995,121047300,23.837380,38.612302,14.146784,Compra,False,,


In [11]:
class Operaciones:
    def __init__(self):
        self.habilitar_stop_loss = False
        self.habilitar_take_profit = False
        self.habilitar_trailing_stop = False
        self.precio_compra = 0
        self.precio_maximo = 0
        self.fecha_compra = None

    def check_stop_loss(self, precio_actual, porcentaje):
        if self.habilitar_stop_loss and self.precio_compra != 0:
            return precio_actual < self.precio_compra * (1 - porcentaje / 100)
        else:
            return False

    def check_take_profit(self, precio_actual, porcentaje):
        if self.habilitar_take_profit and self.precio_compra != 0:
            return (precio_actual - self.precio_compra) / self.precio_compra >= porcentaje / 100
        else:
            return False

    def check_trailing_stop(self, precio_actual, porcentaje):
        if self.habilitar_trailing_stop and self.precio_maximo != 0:
            return precio_actual < self.precio_maximo * (1 - porcentaje / 100)
        else:
            return False



def ejecutar_operaciones(df, acciones_inicial, capital_inicial, comisiones,
                         porcentaje_stop_loss, porcentaje_take_profit, porcentaje_trailing_stop):
    try:
        acciones = acciones_inicial
        capital = capital_inicial
        operaciones = Operaciones()

        # Crear el DataFrame trades_data con las columnas adicionales
        trades_data = pd.DataFrame(columns=['Operacion', 'Fecha de Compra', 'Precio de Compra', 'Cantidad Comprada',
                                            'Capital Inicial', 'Fecha de Venta', 'Tipo de venta', 'Precio de Venta',
                                            'Capital Final', 'Días de Operación', 'Resultado', 'Resultado Porcentual'])

        # Inicializar variables necesarias
        precio_compra = 0
        cantidad_comprada = 0
        precio_maximo = 0
        fecha_compra = None
        operacion = 1

        # Función para calcular días de operación
        def calcular_dias_operacion(fecha_compra, fecha_actual):
            if fecha_compra is not None:
                dias_operacion = (fecha_actual - fecha_compra).days
            else:
                dias_operacion = 0  # Establecer un valor predeterminado o cualquier otra acción deseada
            return dias_operacion

        # Función para registrar una operación en trades_data
        def registrar_operacion(tipo_venta, precio_venta, dias_operacion, resultado_porcentual, resultado, fecha_venta):
            nonlocal operacion
            trades_data.at[operacion, 'Operacion'] = operacion
            trades_data.at[operacion, 'Precio de Venta'] = precio_venta
            trades_data.at[operacion, 'Fecha de Venta'] = fecha_venta
            trades_data.at[operacion, 'Días de Operación'] = dias_operacion
            trades_data.at[operacion, 'Resultado Porcentual'] = resultado_porcentual
            trades_data.at[operacion, 'Resultado'] = resultado
            trades_data.at[operacion, 'Tipo de venta'] = tipo_venta

        # Recorrer el DataFrame de precios
        for row in df.itertuples():
            # Para el primer índice, inicializar acciones y capital
            if row.Index == 0:
                df.at[row.Index, 'Acciones'] = acciones
                df.at[row.Index, 'Capital'] = capital

            else:
                # Caso de compra
                if df.at[row.Index, 'Orden'] == 'Compra':
                    precio_compra = df.at[row.Index, 'Price']
                    precio_maximo = df.at[row.Index, 'Price']
                    cantidad_comprada = (capital - capital * comisiones) / df.at[row.Index, 'Price']
                    acciones += cantidad_comprada
                    fecha_compra = df.at[row.Index, 'Date']

                    # Registrar la transacción en trades_data
                    trades_data = trades_data.append({}, ignore_index=True)
                    trades_data.at[operacion, "Operacion"] = operacion
                    trades_data.at[operacion, "Precio de Compra"] = df.at[row.Index, 'Price']
                    trades_data.at[operacion, "Fecha de Compra"] = df.at[row.Index, 'Date']
                    trades_data.at[operacion, "Cantidad Comprada"] = cantidad_comprada
                    trades_data.at[operacion, "Capital Inicial"] = capital

                    capital = 0

                # Caso de venta
                elif df.at[row.Index, 'Orden'] == 'Venta':
                    precio_venta = df.at[row.Index, 'Price']
                    precio_maximo = 0
                    dias_operacion = calcular_dias_operacion(fecha_compra, df.at[row.Index, 'Date'])
                    capital = acciones * precio_venta * (1 - comisiones)
                    resultado = capital - trades_data.at[operacion, "Capital Inicial"]
                    resultado_porcentual = (precio_venta - precio_compra) * (1 - comisiones) / precio_compra * 100
                    fecha_venta = df.at[row.Index, 'Date']

                    df.at[row.Index, 'Acciones'] = acciones
                    df.at[row.Index, 'Capital'] = capital


                    # Registrar la transacción en trades_data
                    registrar_operacion("Normal", precio_venta, dias_operacion, resultado_porcentual, resultado,
                                        fecha_venta)
                    acciones = 0
                    operacion += 1
                    precio_compra = 0

                else:
                    if acciones > 0:  # Si tenemos acciones compradas
                        precio_maximo = max(precio_maximo, df.at[row.Index, 'maximo'])  # Actualizamos precio_maximo si es necesario

                    if operaciones.check_stop_loss(df.at[row.Index, 'minimo'], porcentaje_stop_loss):
                        precio_venta = df.at[row.Index, 'Price']
                        precio_maximo = 0
                        dias_operacion = calcular_dias_operacion(fecha_compra, df.at[row.Index, 'Date'])
                        capital += acciones * precio_venta * (1 - comisiones)
                        resultado = acciones * ((precio_venta * (1 - comisiones)) - (precio_compra * (1 + comisiones)))
                        resultado_porcentual = resultado / precio_compra * (1 + comisiones) * 100
                        fecha_venta = df.at[row.Index, 'Date']
                        registrar_operacion("Stop Loss", precio_venta, dias_operacion, resultado_porcentual, resultado,
                                            fecha_venta)

                        acciones = 0
                        operacion += 1
                        precio_compra = 0

                    elif operaciones.check_take_profit(df.at[row.Index, 'maximo'], porcentaje_take_profit):
                        precio_venta = df.at[row.Index, 'Price']
                        precio_maximo = 0
                        dias_operacion = calcular_dias_operacion(fecha_compra, df.at[row.Index, 'Date'])
                        capital += acciones * precio_venta * (1 - comisiones)
                        resultado = acciones * (precio_venta - precio_compra) * (1 - comisiones)
                        resultado_porcentual = (precio_venta - precio_compra) * (1 - comisiones) / precio_compra * 100
                        fecha_venta = df.at[row.Index, 'Date']
                        registrar_operacion("Take Profit", precio_venta, dias_operacion, resultado_porcentual, resultado,
                                            fecha_venta)

                        acciones = 0
                        operacion += 1
                        precio_compra = 0

                    elif operaciones.check_trailing_stop(df.at[row.Index, 'Price'], porcentaje_trailing_stop):
                        precio_venta = df.at[row.Index, 'Price']
                        precio_maximo = 0
                        dias_operacion = calcular_dias_operacion(fecha_compra, df.at[row.Index, 'Date'])
                        capital += acciones * precio_venta * (1 - comisiones)
                        resultado = acciones * (precio_venta - precio_compra) * (1 - comisiones)
                        resultado_porcentual = (precio_venta - precio_compra) * (1 - comisiones) / precio_compra * 100
                        fecha_venta = df.at[row.Index, 'Date']
                        registrar_operacion("Trailing Stop", precio_venta, dias_operacion, resultado_porcentual, resultado,
                                            fecha_venta)

                        acciones = 0
                        operacion += 1
                        precio_compra = 0

                    df.at[row.Index, 'Acciones'] = acciones
                    df.at[row.Index, 'Capital'] = capital

        variacion_papel = ((df["Price"].iloc[-1] - df["Price"].iloc[0]) / df["Price"].iloc[0]) * 100
        for row in df.itertuples():
          if df.at[row.Index, 'Capital'] == 0:
            df.at[row.Index, 'capital acumulado']= df.at[row.Index, 'Acciones'] * df.at[row.Index, 'Price']
          else:
            df.at[row.Index, 'capital acumulado']= df.at[row.Index, 'Capital']


        #trades_data.fillna(0, inplace=True)
        trades_data = trades_data.drop(0)
    except KeyError as e:
        print(f"Error: la clave {e} no se encuentra en el DataFrame.")
    except Exception as e:
        print(f"Error inesperado: {e}")

    return df, capital, trades_data


df_resultado, capital_final, trades_data = ejecutar_operaciones(df, acciones_inicial, capital_inicial,comisiones, porcentaje_stop_loss,porcentaje_take_profit,porcentaje_trailing_stop)



In [12]:
995.
trades_data

,Operacion,Fecha de Compra,Precio de Compra,Cantidad Comprada,Capital Inicial,Fecha de Venta,Tipo de venta,Precio de Venta,Capital Final,Días de Operación,Resultado,Resultado Porcentual
1,1,2020-01-24 00:00:00,80.0625,12.427791,1000,2020-02-03 00:00:00,Normal,76.074997,NaN,10,-59.283074,-4.955585
2,2,2020-02-04 00:00:00,78.827499,11.874198,940.716926,2020-02-10 00:00:00,Normal,78.544998,NaN,6,-12.721354,-0.356587
3,3,2020-02-11 00:00:00,80.900002,11.413542,927.995571,2020-02-18 00:00:00,Normal,78.839996,NaN,7,-32.651152,-2.533628
4,4,2020-02-19 00:00:00,80.0,11.135846,895.344419,2020-02-20 00:00:00,Normal,80.657501,NaN,1,-1.645837,0.817767
5,5,2020-04-07 00:00:00,67.699997,13.134862,893.698581,2020-07-24 00:00:00,Normal,90.987503,NaN,108,295.434135,34.2261
6,6,2020-07-27 00:00:00,93.709999,12.626049,1189.132716,2020-09-04 00:00:00,Normal,120.07,NaN,39,319.296951,27.988689
7,7,2020-10-12 00:00:00,120.059998,12.501146,1508.429667,2020-10-19 00:00:00,Normal,119.959999,NaN,7,-16.290431,-0.082874
8,8,2020-11-05 00:00:00,117.949997,12.587355,1492.139236,2020-11-23 00:00:00,Normal,117.18,NaN,18,-24.527849,-0.649552
9,9,2020-11-30 00:00:00,116.970001,12.48417,1467.611387,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df

,Date,Price,maximo,minimo,cierre,volumen,ADX,ADX+,ADX-,Posicion,Alternancia,Orden,Orden_Precio,Acciones,Capital,capital acumulado
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,135480400,0.000000,0.000000,0.000000,Venta,True,Venta,74.05999755859375,0.00000,1000.0,1000.000000
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,146322800,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,118387200,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,108872000,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,132079200,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-12-23,132.160004,132.429993,130.779999,130.960007,88223700,19.541019,34.533669,18.712291,Compra,False,,,12.48417,0.0,1649.907896
248,2020-12-24,131.320007,133.460007,131.100006,131.970001,54930100,20.490430,34.911115,17.652895,Compra,False,,,12.48417,0.0,1639.421239
249,2020-12-28,133.990005,137.339996,133.509995,136.690002,124486200,22.103937,39.235249,15.608724,Compra,False,,,12.48417,0.0,1672.753949
250,2020-12-29,138.050003,138.789993,134.339996,134.869995,121047300,23.837380,38.612302,14.146784,Compra,False,,,12.48417,0.0,1723.439647


In [15]:


class Indicadores:
    # La clase `Indicadores` contiene funciones para calcular varios indicadores financieros.


    def calcular_resultado_operacion(self, trades_data, comisiones):
        # Esta función calcula el resultado de cada operación teniendo en cuenta las comisiones.
        trades_data['Capital Final'] = (trades_data['Precio de Venta']* trades_data['Cantidad Comprada']) * (1 - comisiones)
        return trades_data

    def calcular_resultado_final(self, trades_data, capital_inicial, acciones, df):
        # Esta función calcula el resultado final del trading. Si no queda capital, se calcula en base al precio de las acciones restantes.
        if pd.isnull(trades_data['Precio de Venta'].iloc[-1]):
            df['Date'] = pd.to_datetime(df['Date'])
            trades_data['Fecha de Venta'] = pd.to_datetime(trades_data['Fecha de Venta'], errors='coerce')
            trades_data['Fecha de Compra'] = pd.to_datetime(trades_data['Fecha de Compra'], errors='coerce')
            trades_data['Fecha de Venta'].iloc[-1] = df['Date'].iloc[-1]
            dias_operacion = (trades_data['Fecha de Venta'].iloc[-1] - trades_data['Fecha de Compra'].iloc[-1]).days

            resultadofinal = trades_data['Cantidad Comprada'].iloc[-1] * df["Price"].iloc[-1]
            trades_data['Capital Final'].iloc[-1] = resultadofinal
            trades_data['Tipo de venta'].iloc[-1] = "Fin Backtesting"

            trades_data['Precio de Venta'].iloc[-1] = df['Price'].iloc[-1]
            trades_data['Días de Operación'].iloc[-1] = dias_operacion
            trades_data['Resultado'].iloc[-1] = trades_data['Capital Final'].iloc[-1] - trades_data['Capital Inicial'].iloc[-1]
            trades_data['Resultado Porcentual'].iloc[-1] = (trades_data['Resultado'].iloc[-1]/trades_data['Capital Inicial'].iloc[-1])*100
        else:
            resultadofinal = trades_data['Capital Final'].iloc[-1]
        return round(resultadofinal, 2)



    def calcular_operaciones_positivas(self, trades_data):
        # Esta función calcula el número de operaciones con resultado positivo.
        return trades_data[trades_data['Resultado'] > 0]['Resultado'].count()

    def calcular_mayor_ganancia(self, trades_data):
        # Esta función calcula la mayor ganancia obtenida en una operación.
        return trades_data['Resultado'].max()

    def calcular_mayor_perdida(self, trades_data):
        # Esta función calcula la mayor pérdida obtenida en una operación.
        return trades_data['Resultado'].min()

    def calcular_cagr(self, df):
        # Esta función calcula el CAGR (Compound Annual Growth Rate).
        data = df['capital acumulado']
        years = len(df) / 360
        return (data.iloc[-1] / data.iloc[0]) ** (1 / years) - 1

    def calcular_volatilidad(self, df):
        # Esta función calcula la volatilidad anualizada del capital acumulado.
        return df['capital acumulado'].pct_change().std() * np.sqrt(360)

    def calcular_sharpe_ratio(self, df, risk_free_rate=0.045):
        returns = df['capital acumulado'].pct_change()
        sharpe_ratio = (self.calcular_cagr(df) - risk_free_rate) / self.calcular_volatilidad(df)
        return sharpe_ratio

    def calcular_sortino_ratio(self, df, risk_free_rate=0.045):
        df['retorno_diario'] = df['capital acumulado'].pct_change()
        negative_returns = df[df['retorno_diario'] < 0]['retorno_diario'].std() * np.sqrt(360)
        sortino_ratio = (self.calcular_cagr(df) - risk_free_rate) / negative_returns
        return sortino_ratio

    def calcular_maximo_operaciones_negativas_consecutivas(self, trades_data):
        max_consecutivas = 0
        consecutivas = 0
        for resultado in trades_data['Resultado']:
            if resultado < 0:
                consecutivas += 1
                if consecutivas > max_consecutivas:
                    max_consecutivas = consecutivas
            else:
                consecutivas = 0
        return max_consecutivas


    def calcular_maximo_drawdown(self, df):
        # Esta función calcula el máximo drawdown del capital acumulado.
        data = df['capital acumulado']
        roll_max = data.cummax()
        drawdown = data / roll_max - 1.0
        return drawdown.min()

    def calcular_ratio_ganancia_perdida_transacciones(self, trades_data):
        # Esta función calcula el ratio de ganancias y pérdidas de las operaciones.
        ganancias = trades_data[trades_data['Resultado'] > 0]['Resultado']
        perdidas = trades_data[trades_data['Resultado'] < 0]['Resultado']
        promedio_ganancias = ganancias.mean() if len(ganancias) > 0 else 0
        promedio_perdidas = perdidas.mean() if len(perdidas) > 0 else 0
        ratio_ganancia_perdida = promedio_ganancias / abs(promedio_perdidas) if promedio_perdidas != 0 else float('inf')
        return ratio_ganancia_perdida


    def calcular_retorno_total(self, trades_data):
        # Esta función calcula el retorno total de las operaciones efectivamente hechas.
        retorno_total = trades_data['Resultado'].sum()
        return retorno_total

    def calcular_resultado_porcentual(self, trades_data):
        # Esta función calcula el resultado porcentual de cada operación.
        trades_data['Resultado Porcentual'] = ((trades_data['Precio de Venta'] - trades_data['Precio de Compra']) / trades_data['Precio de Compra']) * 100
        return trades_data

    def calcular_resultado_porcentual_acumulado(self, trades_data):
        # Esta función calcula el resultado porcentual acumulado de todas las operaciones.
        Resultado_Porcentual = trades_data['Resultado Porcentual'].sum()
        return Resultado_Porcentual

    def calcular_ratio_operaciones_positivas(self, trades_data):
        # Esta función calcula el ratio de operaciones con resultado positivo.
        total_operaciones = len(trades_data)
        operaciones_positivas = self.calcular_operaciones_positivas(trades_data)
        ratio_operaciones_positivas = operaciones_positivas / total_operaciones
        return ratio_operaciones_positivas

    def calcular_cantidad_operaciones(self, trades_data):
        # Esta función calcula la cantidad total de operaciones.
        return len(trades_data)

    def calcular_resultado_con_capital_fijo(self, trades_data, capital_fijo):
        # Esta función calcula el resultado de las operaciones con un capital fijo.
        # Suponemos que capital_fijo es el capital dedicado a cada operación individual.
        trades_data['resultado_con_capital_fijo'] = capital_fijo * trades_data['Resultado Porcentual'] / 100
        resultado_con_capital_fijo= trades_data['resultado_con_capital_fijo'].sum()
        return resultado_con_capital_fijo

    def calcular_variacion_papel(self, df):
        # Esta función calcula la variación porcentual del valor de un activo (papel) en el mismo periodo.
        return (df['Price'].iloc[-1] - df['Price'].iloc[0]) / df['Price'].iloc[0] * 100

    def calcular_dias_sin_operar(self, trades_data, df):
        # Esta función calcula el número de días sin operaciones.
        df['Date'] = pd.to_datetime(df['Date'])
        trades_data['Fecha de Venta'] = pd.to_datetime(trades_data['Fecha de Venta'], errors='coerce')
        trades_data['Fecha de Compra'] = pd.to_datetime(trades_data['Fecha de Compra'], errors='coerce')

        dias_operaciones = trades_data['Días de Operación'].sum()
        dias_totales = df['Date'].dt.dayofyear.unique().tolist()
        return len(dias_totales) - (dias_operaciones)

    def calcular_ratio_mayor_perdida_ganancia(self, trades_data):
        # Esta función calcula el ratio entre la mayor pérdida y la mayor ganancia.
        mayor_perdida = self.calcular_mayor_perdida(trades_data)
        mayor_ganancia = self.calcular_mayor_ganancia(trades_data)
        ratio_perdida_ganancia = mayor_perdida / mayor_ganancia if mayor_ganancia != 0 else float('inf')
        return ratio_perdida_ganancia


    def calcular_ratio_sortino_modificado(self, trades_data, umbral_objetivo_riesgo):
        # Esta función calcula el Ratio de Sortino Modificado.
        trades_data['retorno_diario'] = trades_data['Resultado'] / trades_data['Capital Inicial']
        trades_data['retorno_negativo'] = trades_data['retorno_diario'].apply(lambda x: x if x < 0 else 0)
        desviacion_negativa = trades_data['retorno_negativo'].std()
        ratio_sortino_modificado = (self.calcular_retorno_total(trades_data) - umbral_objetivo_riesgo) / desviacion_negativa
        return ratio_sortino_modificado

    def calcular_ratio_calmar(self, df):
        # Esta función calcula el Ratio de Calmar.
        cagr = self.calcular_cagr(df)
        max_drawdown = self.calcular_maximo_drawdown(df)
        ratio_calmar = cagr / abs(max_drawdown)
        return ratio_calmar

    def calcular_drawdowns(self, df):
    # Esta función calcula los drawdowns a partir de los datos del capital acumulado.
        data = df['capital acumulado']
        roll_max = data.cummax()
        drawdowns = data - roll_max
        return drawdowns


    def calcular_ratio_ulcer(self, df):
        # Esta función calcula el Ratio de Ulcer.
        drawdowns = self.calcular_drawdowns(df)
        ulcer_index = np.sqrt(np.mean(drawdowns ** 2))
        return ulcer_index

    def calcular_indice_modificacion_sharpe(self, df, risk_free_rate=0.045):
        # Esta función calcula el Índice de Modificación de Sharpe.
        cagr = self.calcular_cagr(df)
        volatilidad = self.calcular_volatilidad(df)
        max_drawdown = self.calcular_maximo_drawdown(df)
        sharpe_ratio = self.calcular_sharpe_ratio(df, risk_free_rate=risk_free_rate)
        indice_modificacion_sharpe = (cagr - risk_free_rate) / (volatilidad * abs(max_drawdown) * sharpe_ratio)
        return indice_modificacion_sharpe


    # Esta función imprime los resultados de los indicadores calculados anteriormente.

    def imprimir_resultados(self, trades_data, comisiones, capital_final, acciones, df_resultado, risk_free_rate=4):
        resultado_operacion = self.calcular_resultado_operacion(trades_data, comisiones)
        resultado_final = self.calcular_resultado_final(trades_data, capital_inicial, acciones, df_resultado)
        operaciones_positivas = self.calcular_operaciones_positivas(trades_data)
        mayor_ganancia = self.calcular_mayor_ganancia(trades_data)
        mayor_perdida = self.calcular_mayor_perdida(trades_data)
        cagr = self.calcular_cagr(df_resultado)
        volatilidad = self.calcular_volatilidad(df_resultado)
        sharpe_ratio = self.calcular_sharpe_ratio(df_resultado, risk_free_rate=risk_free_rate)
        sortino_ratio = self.calcular_sortino_ratio(df_resultado, risk_free_rate=risk_free_rate)
        maximo_drawdown = self.calcular_maximo_drawdown(df_resultado)
        ratio_ganancia_perdida = self.calcular_ratio_ganancia_perdida_transacciones(trades_data)
        retorno_total = self.calcular_retorno_total(trades_data)
        resultado_porcentual = self.calcular_resultado_porcentual(trades_data)
        resultado_porcentual_acumulado = self.calcular_resultado_porcentual_acumulado(trades_data)
        ratio_operaciones_positivas = self.calcular_ratio_operaciones_positivas(trades_data)
        cantidad_operaciones = self.calcular_cantidad_operaciones(trades_data)
        resultado_con_capital_fijo = self.calcular_resultado_con_capital_fijo(trades_data, capital_fijo)  # Modificar aquí
        variacion_papel = self.calcular_variacion_papel(df_resultado)
        dias_sin_operar = self.calcular_dias_sin_operar(trades_data, df)
        ratio_perdida_ganancia= self.calcular_ratio_mayor_perdida_ganancia(trades_data)
        max_operaciones_negativas_consecutivas = self.calcular_maximo_operaciones_negativas_consecutivas(trades_data)
        umbral_objetivo_riesgo = 0.02  # Umbral objetivo de riesgo: 2%
        ratio_sortino_modificado = indicadores.calcular_ratio_sortino_modificado(trades_data, umbral_objetivo_riesgo)
        ratio_calmar = indicadores.calcular_ratio_calmar(df_resultado)
        ratio_ulcer = indicadores.calcular_ratio_ulcer(df_resultado)
        indice_modificacion_sharpe = indicadores.calcular_indice_modificacion_sharpe(df_resultado)







        print("\nRESULTADOS BACKTESTING\n")
        print(f"Resultado Final: {resultado_final}")
        print(f"Total de Operaciones: {len(trades_data)}")
        print(f"Operaciones Positivas: {operaciones_positivas}")
        print(f"Mayor Ganancia: {mayor_ganancia}")
        print(f"Mayor Pérdida: {mayor_perdida}")
        print(f"CAGR: {cagr * 100:.2f}%")
        print(f"Volatilidad: {volatilidad:.2f}")
        print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
        print(f"Sortino Ratio: {sortino_ratio:.2f}")
        print(f"Máximo Drawdown: {maximo_drawdown * 100:.2f}%")
        print(f"Ratio Ganancia/Perdida: {ratio_ganancia_perdida:.2f}")
        print(f"Ratio Mayor Ganancia/Peor perdida: {ratio_perdida_ganancia}" + ", si el ratio es 0.2, significa que la mayor pérdida es solo el 20% de la mayor ganancia. Esto indica una buena gestión de riesgos y un potencial de ganancias considerablemente mayor que las pérdidas.")
        print(f"Ratio Operaciones Positivas: {ratio_operaciones_positivas}")
        print(f"Retorno Neto Total: {retorno_total}")
        print(f"Resultado Porcentual Acumulado: {resultado_porcentual_acumulado}")
        print(f"Cantidad de operaciones: {cantidad_operaciones}")
        print(f"El resultado de la operación con capital fijo por operación sería: ${resultado_con_capital_fijo}")
        print(f"La variación porcentual en el mismo período del papel fue: {variacion_papel}%")
        print(f"La cantidad de días sin operar del papel fue: {dias_sin_operar} sobre {len(df_resultado.index)}")
        print(f"Máximo de operaciones negativas consecutivas: {max_operaciones_negativas_consecutivas}")
        print(f"Ratio de Sortino Modificado: {ratio_sortino_modificado}")
        # Guía: Un número alto indica un buen rendimiento ajustado al riesgo, mientras que un número bajo indica un rendimiento insuficiente en comparación con el riesgo asumido.
        print(f"Ratio de Calmar: {ratio_calmar}")
        # Guía: Un número alto indica un buen rendimiento ajustado al drawdown, mientras que un número bajo indica un rendimiento insuficiente en comparación con el drawdown.
        print(f"Ratio de Ulcer: {ratio_ulcer}")
        # Guía: Un número bajo indica una baja volatilidad y una baja presencia de drawdowns, mientras que un número alto indica una alta volatilidad y una mayor presencia de drawdowns.
        print(f"Índice de Modificación de Sharpe: {indice_modificacion_sharpe}")
        # Guía: Un número alto indica un mejor rendimiento ajustado al riesgo y drawdown en comparación con el Índice de Sharpe estándar.


df_resultado['Date'] = pd.to_datetime(df_resultado['Date'])

indicadores = Indicadores()


indicadores.imprimir_resultados(trades_data, comisiones, capital_final, acciones,  df_resultado, capital_fijo)


RESULTADOS BACKTESTING

Resultado Final: 1684.14
Total de Operaciones: 9
Operaciones Positivas: 3
Mayor Ganancia: 319.29695070122807
Mayor Pérdida: -59.283074427060455
CAGR: 112.08%
Volatilidad: 0.36
Sharpe Ratio: -2743.87
Sortino Ratio: -3161.17
Máximo Drawdown: -16.28%
Ratio Ganancia/Perdida: 11.42
Ratio Mayor Ganancia/Peor perdida: -0.18566752453120888, si el ratio es 0.2, significa que la mayor pérdida es solo el 20% de la mayor ganancia. Esto indica una buena gestión de riesgos y un potencial de ganancias considerablemente mayor que las pérdidas.
Ratio Operaciones Positivas: 0.3333333333333333
Retorno Neto Total: 692.6037332225519
Resultado Porcentual Acumulado: 70.63803243214167
Cantidad de operaciones: 9
El resultado de la operación con capital fijo por operación sería: $706.3803243214167
La variación porcentual en el mismo período del papel fue: 83.06779138601564%
La cantidad de días sin operar del papel fue: 26 sobre 252
Máximo de operaciones negativas consecutivas: 4
Ratio d

In [16]:
df


,Date,Price,maximo,minimo,cierre,volumen,ADX,ADX+,ADX-,Posicion,Alternancia,Orden,Orden_Precio,Acciones,Capital,capital acumulado,retorno_diario
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,135480400,0.000000,0.000000,0.000000,Venta,True,Venta,74.05999755859375,0.00000,1000.0,1000.000000,NaN
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,146322800,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000,0.000000
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,118387200,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000,0.000000
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,108872000,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000,0.000000
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,132079200,0.000000,0.000000,0.000000,Venta,False,,,0.00000,1000.0,1000.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-12-23,132.160004,132.429993,130.779999,130.960007,88223700,19.541019,34.533669,18.712291,Compra,False,,,12.48417,0.0,1649.907896,0.004179
248,2020-12-24,131.320007,133.460007,131.100006,131.970001,54930100,20.490430,34.911115,17.652895,Compra,False,,,12.48417,0.0,1639.421239,-0.006356
249,2020-12-28,133.990005,137.339996,133.509995,136.690002,124486200,22.103937,39.235249,15.608724,Compra,False,,,12.48417,0.0,1672.753949,0.020332
250,2020-12-29,138.050003,138.789993,134.339996,134.869995,121047300,23.837380,38.612302,14.146784,Compra,False,,,12.48417,0.0,1723.439647,0.030301
